In [ ]:
import os
import sys
import json
import tempfile
import pandas as pd

from threading import Thread
from CoolProp.CoolProp import PropsSI

# shortcuts
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

sys.path.append(norm(join(cwd, '..', '..', '..', 'glhe')))
sys.path.append(norm(join(cwd, '..', '..', '..', 'standalone')))

from standalone.run_g_function import RunGFunctions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('seaborn-bright')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 12

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
df_exp = pd.read_csv('MFRTRT_loads.csv', parse_dates=True, index_col='Date/Time')

In [ ]:
df_exp.head(1)

In [ ]:
timestep = 1 # minutes
df_exp = df_exp.resample('{}T'.format(timestep)).mean()
df_exp = df_exp.interpolate(method='linear')

In [ ]:
df_exp.head(3)

In [ ]:
df_exp.plot(y=['mdot [kg/s]'])

In [ ]:
df_exp.plot(y=['Meas. Total Power [W]'])

In [ ]:
df_exp.to_csv('in.csv', columns=['Meas. Total Power [W]', 'mdot [kg/s]', 'Outlet 1 [C]'])

In [ ]:
df_exp['Run Time'] = df_exp.index.to_series().diff().dt.total_seconds().cumsum()
df_exp['Run Time'].iloc[0] = 0
df_exp['Run Time'] = df_exp['Run Time'].astype('int')
df_exp.set_index('Run Time', drop=True, inplace=True)

In [ ]:
in_file = join(cwd, 'MFRTRT_input.json')
with open(in_file, 'r') as f:
    json_blob = f.read()
in_json = json.loads(json_blob)

In [ ]:
in_json['simulation']['runtime'] = 190 * 3600
in_json['simulation']['time-steps per hour'] = int(60 / timestep)

In [ ]:
load_file = in_json['load-profile']['external']['path'] 
in_json['load-profile']['external']['path'] = join(cwd, load_file)

flow_file = in_json['flow-profile']['external']['path']
in_json['flow-profile']['external']['path'] = join(cwd, flow_file)

g_file = in_json['g-functions']['file']
in_json['g-functions']['file'] = join(cwd, g_file)

In [ ]:
in_json['load-aggregation']['type'] = 'dynamic'
in_json['simulation']['output-path'] = join(cwd, 'out.csv')
in_json['simulation']['plant driver'] = 'inlet-flow'

in_json['load-aggregation']['dynamic'] = {}
in_json['load-aggregation']['dynamic']['runtime'] = in_json['simulation']['runtime'] = in_json['simulation']['runtime']

with open(join(cwd, 'in.json'), 'w') as f:
    f.write(json.dumps(in_json, sort_keys=True, indent=2))

In [ ]:
in_path = os.path.join(os.getcwd(), 'in.json')

In [ ]:
def do_a_run(in_file):
    RunGFunctions(in_file).simulate()

In [ ]:
p2 = Thread(target=do_a_run, args=(in_path,))

p2.start()

p2.join()

In [ ]:
df_model = pd.read_csv("out.csv", index_col='Simulation Time')

In [ ]:
start_pt = 0
end_pt = 680000

In [ ]:
# fig = plt.figure(figsize=(7,5))
fig = plt.figure()

in_exp = df_exp['Outlet 1 [C]'].loc[start_pt : end_pt]
out_exp = df_exp['Inlet 1 [C]'].loc[start_pt : end_pt]
plt.plot(in_exp, label='Inlet (Exp)', marker='x', markevery=100)
plt.plot(out_exp, label='Outlet (Exp)', marker='x', markevery=100)

in_plant = df_model['Plant Outlet Temperature [C]'].loc[start_pt : end_pt]
out_plant = df_model['Plant Inlet Temperature [C]'].loc[start_pt : end_pt]
plt.plot(in_plant, label='Inlet from Plant', linestyle='--')
plt.plot(out_plant, label='Outlet from Plant', linestyle='--')

in_ghe = df_model['GHE Inlet Temp (After Pipe) [C]'].loc[start_pt : end_pt]
out_ghe = df_model['GHE Outlet Temp (After Pipe) [C]'].loc[start_pt : end_pt]
plt.plot(in_ghe, label='GHE Inlet After Pipe', linestyle='-.')
plt.plot(out_ghe, label='GHE Outlet After Pipe', linestyle='-.')

out_ghe_before = df_model['GHE Outlet Temp (Before Pipe) [C]'].loc[start_pt : end_pt]
plt.plot(out_ghe_before, label='GHE Outlet Before Pipe', linestyle=':')

plt.xlabel('Simulation Time [s]')
plt.ylabel('Temperature [C]')

plt.legend()
plt.grid(True)
# plt.savefig('Temps_1hr_1minTS.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# fig = plt.figure(figsize=(7,5))
fig = plt.figure()

out_exp = df_exp['Inlet 1 [C]'].loc[start_pt : end_pt]
plt.plot(in_exp, label='Inlet (Exp)', marker='x', markevery=10)

in_plant = df_model['Plant Outlet Temperature [C]'].loc[start_pt : end_pt]
plt.plot(in_plant, label='Inlet (From Plant)', linestyle='--')

in_ghe = df_model['GHE Inlet Temp (After Pipe) [C]'].loc[start_pt : end_pt]
plt.plot(in_ghe, label='Inlet (After Pipe)', linestyle='-.')

plt.xlabel('Simulation Time [s]')
plt.ylabel('Temperature [C]')

plt.legend()
plt.grid(True)
# plt.savefig('Temps_1hr_1minTS.pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(7,5))
# fig = plt.figure()
plt.plot(df_model['Flow Fraction [-]'].loc[start_pt : end_pt])
plt.xlabel('Simulation Time [s]')
plt.ylabel('Flow Fraction [-]')
plt.grid(True)

# plt.savefig('FF_192hr_1minTS.pdf', bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(df_exp['Meas. Total Power [W]'].loc[start_pt : end_pt], label="Exp Data", marker='x')
plt.plot(df_model['GLHE Heat Transfer Rate [W]'].loc[start_pt : end_pt], label="Model")

plt.legend()
plt.show()

In [ ]:
plt.plot(df_model['Load on GHE [W/m]'].loc[start_pt : end_pt], label='Model') 

plt.legend()
plt.show()

In [ ]:
df_exp['MFT'] = df_exp[['Inlet 1 [C]', 'Outlet 1 [C]']].mean(axis=1)

In [ ]:
plt.plot(df_model['Average Fluid Temp [C]'].loc[start_pt : end_pt], label="Model")
plt.plot(df_exp['MFT'].loc[start_pt : end_pt], label="Exp", marker='x')

plt.legend()
plt.show()

In [ ]:
plt.plot(df_model["Local Borehole Resistance 'Rb' [K/(W/m)]"].loc[start_pt : end_pt], label="Model")

plt.legend()
plt.show()

In [ ]:
plt.plot(df_model["Soil Resistance 'Rs' [K/(W/m)]"].loc[start_pt : end_pt], label="Model")

plt.legend()
plt.show()